In [2]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

In [3]:
# Tool Create
@tool
def multiply(a:int,b:int):
    """Given 2 numbers a and b this tool returns their product"""
    return a*b

In [4]:
print(multiply.invoke({'a':5,'b':6}))

30


In [5]:
multiply.name

'multiply'

In [6]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [7]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [8]:
# tool binding
model = ChatGroq(model='openai/gpt-oss-120b')

In [9]:
model.invoke('hi')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hi". We need to respond appropriately. It\'s a simple greeting. We can reply friendly.'}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 72, 'total_tokens': 113, 'completion_time': 0.084477516, 'completion_tokens_details': {'reasoning_tokens': 23}, 'prompt_time': 0.002667889, 'prompt_tokens_details': None, 'queue_time': 0.054916331, 'total_time': 0.087145405}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_626f3fc5e0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b918c-6420-7430-b25e-caabfbe42b74-0', usage_metadata={'input_tokens': 72, 'output_tokens': 41, 'total_tokens': 113, 'output_token_details': {'reasoning': 23}})

In [10]:
llm_with_tools = model.bind_tools([multiply])

In [11]:
llm_with_tools.invoke('hi how are you')

AIMessage(content="Hello! I'm doing great, thanks for asking. How can I assist you today?", additional_kwargs={'reasoning_content': 'The user says "hi how are you". Simple greeting. Should respond friendly.'}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 133, 'total_tokens': 176, 'completion_time': 0.089432534, 'completion_tokens_details': {'reasoning_tokens': 17}, 'prompt_time': 0.005041574, 'prompt_tokens_details': None, 'queue_time': 0.054784516, 'total_time': 0.094474108}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_4a19b1544c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b918c-68b8-7182-869a-a7e0680e47cf-0', usage_metadata={'input_tokens': 133, 'output_tokens': 43, 'total_tokens': 176, 'output_token_details': {'reasoning': 17}})

In [12]:
query = HumanMessage('can you multiply 3 with 1000')

In [13]:
messages = [query]

In [14]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [15]:
result = llm_with_tools.invoke(messages)

In [16]:
messages.append(result)

In [17]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants multiplication of 3 and 1000. Simple. Use tool multiply.', 'tool_calls': [{'id': 'fc_dfc3b526-673f-4d40-8ec1-5b268ea5b099', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 138, 'total_tokens': 190, 'completion_time': 0.108464508, 'completion_tokens_details': {'reasoning_tokens': 18}, 'prompt_time': 0.005390091, 'prompt_tokens_details': None, 'queue_time': 0.049861379, 'total_time': 0.113854599}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b918c-73b1-7d71-a635-fc23e32e06ea-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_df

In [18]:
tool_result = multiply.invoke(result.tool_calls[0])

In [19]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='fc_dfc3b526-673f-4d40-8ec1-5b268ea5b099')

In [20]:
messages.append(tool_result)

In [21]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants multiplication of 3 and 1000. Simple. Use tool multiply.', 'tool_calls': [{'id': 'fc_dfc3b526-673f-4d40-8ec1-5b268ea5b099', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 138, 'total_tokens': 190, 'completion_time': 0.108464508, 'completion_tokens_details': {'reasoning_tokens': 18}, 'prompt_time': 0.005390091, 'prompt_tokens_details': None, 'queue_time': 0.049861379, 'total_time': 0.113854599}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b918c-73b1-7d71-a635-fc23e32e06ea-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_df

In [22]:
llm_with_tools.invoke(messages).content

'The product of\u202f3\u202fand\u202f1000\u202fis\u202f**3000**.'

In [23]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
 
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """ 
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()
@tool
def convert(base_currency_value: int,conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [24]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [30]:
import requests
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1767657601,
 'time_last_update_utc': 'Tue, 06 Jan 2026 00:00:01 +0000',
 'time_next_update_unix': 1767744001,
 'time_next_update_utc': 'Wed, 07 Jan 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.2979}

In [32]:
convert.invoke({'base_currency_value':10,'conversion_rate':85.16})

851.5999999999999

In [33]:
# tool binding
model = ChatGroq(model = 'openai/gpt-oss-120b')

In [34]:
llm_with_tools = model.bind_tools([get_conversion_factor,convert])

In [35]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [36]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [37]:
ai_message = llm_with_tools.invoke(messages)

In [38]:
messages.append(ai_message)

In [40]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'fc_a1e0b92b-8594-40c0-9a3f-5e2d3a7f598d',
  'type': 'tool_call'}]

In [41]:
import json
for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [42]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to get conversion factor between INR and USD. Use get_conversion_factor with base_currency "INR", target_currency "USD". Then use convert to convert 10 INR to USD. The convert function expects base_currency_value, but likely also needs the conversion factor? The description says "given a currency conversion rate this function calculates the target currency value from a given base currency value". But we need to pass base_currency_value only? Possibly the function uses previously fetched factor? Might need to pass both? The signature only has base_currency_value. Probably the function uses stored factor? We\'ll call get_conversion_factor first, then convert.', 'tool_calls': [{'id': 'fc_a1e0b92b-8594-40c0-9a3f-5e2d3a7f598d', 'function': {'arguments': 

In [46]:
llm_with_tools.invoke(messages).content

'The current conversion factor from **Indian Rupees (INR) to US Dollars (USD)** is:\n\n**1\u202fINR\u202f≈\u202f0.01107\u202fUSD**\n\nUsing this rate:\n\n\\[\n10 \\text{ INR} \\times 0.01107 \\frac{\\text{USD}}{\\text{INR}} \\;=\\; 0.1107 \\text{ USD}\n\\]\n\n**So, 10\u202fINR is approximately\u202f0.11\u202fUSD.**'

In [49]:
from langchain.agents import create_agent

In [50]:
agent_executor = create_agent(
    tools = [get_conversion_factor,convert],
    llm=model,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

NameError: name 'AgentType' is not defined